# First Steps with Prompt Engineering

In [ ]:
import os
from openai import OpenAI
!pip install cohere
import cohere

In [ ]:
from google.colab import userdata


co = cohere.Client(userdata.get('COHERE_API_KEY'))

openai_client = OpenAI(
    api_key=userdata.get("OPENAI_API_KEY")
)

In [ ]:
def test_prompt_openai(prompt, suppress=False, model='gpt-3.5-turbo', **kwargs):
    " a simple function to take in a prompt and run it through a given model "

    chat_completion = openai_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        **kwargs
    )
    answer = chat_completion.choices[0].message.content
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer


In [ ]:
def test_prompt_cohere(prompt, suppress=False, model='command-a-03-2025', **kwargs):
    response = co.chat(
        message=prompt,
        model=model,
        **kwargs,
      )
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{response.text}')
    else:
        return response.text

## Just ASK

In [ ]:
test_prompt_openai('Translate to Turkish.\n\nWhere is the nearest restaurant?')

In [ ]:
test_prompt_cohere('Translate to Turkish.\n\nWhere is the nearest restaurant?')

In [ ]:
# depending on the capability of the model, you may need to coax it to structure the output better
# Not the best Turkish..
test_prompt_cohere('Translate to Turkish.\n\nEnglish: Where is the nearest restaurant?\nTurkish:')

# Few-shot learning

Using examples to "teach" GPT-3 what to do

## The original GPT-3 paper was called:
![gpt3_paper.png](https://github.com/dmoneyUK/quick-start-guide-to-llms/blob/main/images/gpt3_paper.png?raw=1)

In [ ]:
examples = [
    ('Review: This movie sucks\nSubjective: Yes'),
    ('Review: This tv show was about the ocean\nSubjective: No'),
    ('Review: This book had a lot of flaws\nSubjective: Yes'),

    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

In [ ]:
# Cohere is getting this example right
test_prompt_cohere('\n###\n'.join(examples))  # ### is a common few-shot separator

In [ ]:
# Without the examples:
test_prompt_openai('Review: The book was about WWII\nSubjective:')

In [ ]:
# With a prompt
test_prompt_openai('Tell me the subjectivity of this review.\n\nReview: The book was about WWII\nSubjective:')

In [ ]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The book was about WWII\nSubjective:')

In [ ]:
# A different review
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No".\n\nReview: The fight scenes were the best part!\nSubjective:')

In [ ]:
# Be more specific about the output
test_prompt_openai('Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.\n\nReview: The book was about WWII\nSubjective:')


In [ ]:
examples = [
    ('Review: This movie sucks\nSubjective: {"answer": true}'),
    ('Review: This tv show was about the ocean\nSubjective: {"answer": false}'),
    ('Review: This book had a lot of flaws\nSubjective: {"answer": true}'),

    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

# Personas / Style

In [ ]:
# It only takes a few words to pretty drastically change the output

In [ ]:
style = 'rude'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


In [ ]:
style = 'friendly'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


In [ ]:
style = 'yoda'
test_prompt_openai(f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:')


# What a good time to talk about output validation and bias!

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
sequence_to_classify = "All of this is your fault for being Turkish. You shouldn't be allowed to have an account on this website."

candidate_labels = ['racist', 'sexist', 'rude']

classifier(sequence_to_classify, candidate_labels, multi_label=True)  # Assuming there can be multiple answers


In [ ]:
# then the "rude" AI wasn't that bad
classifier(
    'Do you have your login information? Because if not, then there is nothing I can do for you.',
    candidate_labels, multi_label=True)



In [ ]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=0,  # technically (mathematically) we cannot set temp to 0 so OpenAI is doing something under the hood here
        suppress=True
    ))


In [ ]:
# only 1 unique response
responses, len(set(responses))

In [ ]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        suppress=True
    ))
# all different
responses, len(set(responses))


In [ ]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=2,
        suppress=True,
        max_tokens=50
    ))
# all different and horrible responses!
responses, len(set(responses))


In [ ]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=.1,
        suppress=True
    ))
# all different and horrible responses!
responses, len(set(responses))


In [ ]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        top_p=.1,

        suppress=True
    ))
# restricting top p allows fewer tokens to be considered, making the model more deterministic
responses, len(set(responses))
